In [23]:
#Taufik Kurahman Sasmita
#1103200008
#Model CNN Modern VGG

##**Library**

In [1]:
!pip install torch torchvision
!pip install d2l

##**Library**

In [8]:
import torch
import torchvision
from d2l import torch as d2l
from torchvision import transforms
from torch import nn


##**Definisikan Block VGG**

In [11]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

##**Definisikan Arsitektur CGG**

In [18]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1  # FashionMNIST memiliki 1 channel (grayscale)
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096),  # Sesuaikan ukuran input linear layer sesuai dengan output terakhir conv layer
        nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )
    conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512))
net = vgg(conv_arch)


##**Buatkan Model dan Definisikan loss dan optimizer**

In [19]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.05)

##**Persiapkan Data dan DataLoader**

In [20]:
import torchvision
from torchvision import transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_data = torchvision.datasets.FashionMNIST(root='../data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.FashionMNIST(root='../data', train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)


##**Training dan Evaluasi Model**

In [22]:
import torch
from torch import nn
import torch.nn.functional as F

class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))  # Ubah ke AdaptiveAvgPool2d untuk menyesuaikan ukuran output
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # Sesuaikan input layer linear dengan output dari avgpool
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Inisialisasi model
net = VGG()

# Contoh penggunaan pada FashionMNIST
input_data = torch.randn(1, 1, 28, 28)  # Contoh input FashionMNIST
output = net(input_data)
print(output.shape)  # Cetak ukuran output untuk memastikan validitas

# Output: torch.Size([1, 10]) jika menggunakan contoh ini


torch.Size([1, 10])
